#### Loading and preparing the PCam data for training shallow learning models using tensorflow dataset (tfds)

In [1]:
from sklearn import svm
from sklearn.metrics import accuracy_score
import numpy as np

import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.preprocessing import StandardScaler

2023-01-24 15:27:35.434005: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/gustavchristensen/Documents/SDU/MSc. Data Science/3. Semester - DT/Anvendt Maskinlæring/applied_ML_faelles/assignment_2/.venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Defining a function that grayscale, resize and flattens the image. This function might also become handy (for deep learning) if the original images are too large for your hardware configuration.

In [3]:
def convert_sample(image):
    image = tf.image.rgb_to_grayscale(image)
    image = tf.image.resize(image,[32,32]).numpy()
    image = image.reshape(1,-1)
    return image

Next we use the tensorflow dataset API - tfds - to load data from your mounted google drive. Note this API requite that you should have copied the entire **patch_camelyon** folder from https://syddanskuni-my.sharepoint.com/:f:/g/personal/cmd_sam_sdu_dk/EiWD2LmuxCJBp-_tfGK7aL8Bair7l5z8FU5sp5pLjlhKwg?e=FLzWno to the /content/drive/MyDrive folder on your google drive:

In [9]:
ds1,ds2,ds3 = tfds.load('patch_camelyon',
                        split=['train[:10%]','test[:10%]','validation[:10%]'],
                        data_dir = '/Users/gustavchristensen/Documents/SDU/MSc. Data Science/3. Semester - DT/Anvendt Maskinlæring',
                        download=False,
                        batch_size=-1, # All data...no batches needed 
                        as_supervised=True, # So that we easily can transform data to numpy format
                        shuffle_files=True)
                        

Next we can easily convert both the images and the labels to numpy format 

In [10]:
train_dataset       = tfds.as_numpy(ds1)
train_dataset_image = np.vstack(list(map(convert_sample,train_dataset[0])))
train_dataset_image_Scaled = StandardScaler(with_mean=0, with_std=1).fit_transform(train_dataset_image)
train_dataset_label = train_dataset[1].reshape(-1,)    
print(f'Shape of training data features (observations,features): {train_dataset_image_Scaled.shape}')
print(f'Shape of training data labels (observations,): {train_dataset_label.shape}')

validation_dataset  = tfds.as_numpy(ds3)
validation_dataset_image = np.vstack(list(map(convert_sample,validation_dataset[0])))
validation_dataset_image_Scaled = StandardScaler(with_mean=0, with_std=1).fit_transform(validation_dataset_image)
validation_dataset_label = validation_dataset[1].reshape(-1,) 
   
test_dataset        = tfds.as_numpy(ds2)
test_dataset_image = np.vstack(list(map(convert_sample,test_dataset[0])))
test_dataset_image_Scaled = StandardScaler(with_mean=0, with_std=1).fit_transform(test_dataset_image)
test_dataset_label = test_dataset[1].reshape(-1,)

Shape of training data features (observations,features): (20972, 1024)
Shape of training data labels (observations,): (20972,)


The data is then ready to be applied for training, validation, testing in a shallow learning model such as the SVM classifier...below just a very very simple illustration on how to construct and train a support vector machine based on the data we have prepared

In [6]:
clf = svm.SVC(kernel='rbf')
clf.fit(train_dataset_image_Scaled, train_dataset_label)
y_test_hat = clf.predict(test_dataset_image)

# Obtain accuracy by using the `accuracy_score` function
accuracy_linear = accuracy_score(y_test_hat, test_dataset_label )
# Print results
print(f'SVM achieved {round(accuracy_linear * 100, 1)}% accuracy.')

SVM achieved 51.9% accuracy.
